# Unsupervised learning used in the context of exploratory data analysis (module 1 of the CAS in advanced machine learning, university of Bern)

## Credit:
#### Data:
Dr. med. R. Gonzenbach (neuro-rehabilitation clinic Valens) was so generous to provide the data for this analysis. 
#### Feature extraction and data aggregation:
Feature extraction from the raw data (data from movement sensors worn by patients) was performed by an algorithm developped at EPFL by Gaëlle Prigent et al. (2023): 
https://link.springer.com/article/10.1007/s11517-023-02826-x

## Description of the data and the project: 
#### Data recording and aggregation:
The present data are aggregated data from movement sensors worn by patients that underwent neuro-rehabilitation treatment. The walking detection algorithm identified walking bouts. Then average descriptors of the walking bouts such as mean speed, mean stride length etc. were calculated per patient before and after neuro-rehabilitation treatment. The raw data was recorded on several days for periods of 12 hours but only segments of 8 hours (starting from the first movement detected) were provided in order to have the same data length for all patients. 
For details see Gaëlle Prigent et al. (2024): 
https://jneuroengrehab.biomedcentral.com/articles/10.1186/s12984-024-01383-0

Data from different days were pooled for each patient to mean values and standard deviations of the variables in question (speed, stride length, cadence, walking bout time, etc). For the present project I only use these aggregated variables (no raw data).

#### Objective
The goal of this project is to assess if movement patterns changed over the course of the from morning to evening. While machine learning is unlikely to be necessary or useful for the final analysis, it can be useful in data exploration. I use k-means clustering here, to see if there are any interesting groups of patients sharing interesting characteristics. 

In [ ]:
import pandas as pd
import numpy as np
import scipy.io as sio
import os
import re
import matplotlib.pyplot as plt
import source.transform_data as trs
import source.check_data as chd
from pathlib import Path
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist
from sklearn.cluster import AgglomerativeClustering 
from sklearn.cluster import KMeans 

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

## Load data:

In [ ]:
path_to_data = Path('Data')

In [ ]:
data_wide = pd.read_csv(path_to_data/'data_table_wide.csv', index_col=None)

#### For every patient (subject) data from two files were extracted. One file contains the data from before neuro-rehabilitation (T2) and one the data from afterwards (T3). The present data frame is in wide format, i.e. every column contains a variable. 

In [ ]:
data_wide.head()

### Number of walking bouts before vs after rehabilitation treatment:

In [ ]:
data_wide.value_counts('exp_phase_id')

In [ ]:
data_wide.value_counts('exp_phase_descr')

### It looks like for most subjects we have measurements during a period of 8 hours:

In [ ]:
ax = plt.subplot()
scatter = ax.scatter(data_wide.time_stamps_hours, data_wide.speed_mean, alpha=0.4)
ax.set_xlabel("hours after beginning of recording")
ax.set_ylabel("mean speed during walking bout")

#### Is it appropriate to only consider walking bouts within 8 hours and disregard later walking bouts? 
This is only appropriate if patients did not switch of the movement sensors prematurely. If for every patient we have indeed a measuring period of 8 hours, we could subdivide the 8 hours or recording into two phases of 4 hours each (morning and afternoon). The above plot (as well as the methods section in Gaëlle Prigent et al. (2024) suggests that this is the case. Nonetheless, I shall verify this further: 

### Verify if for every patient there is activity throughout the 8 hours time span:

In [ ]:
len(set(data_wide.subject))

#### Assign bins and labels to one hour time spans:

In [ ]:
data_wide["time_stamps_bins"] = pd.cut(data_wide["time_stamps_hours"],
                               bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, np.inf],
                               labels=[1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
data_wide["time_stamps_bins"].dtype

#### Count unique patients per time bin:

In [ ]:
time_bins = list(set(data_wide["time_stamps_bins"]))
subjects_per_hour = chd.count_unique_subjects_per_hour(data_wide, time_bins)

In [ ]:
subjects_per_hour

Hours 4 and 7 had only 42 subjects, during every other hour during the 8 hours time span contains all 43 subjects.

#### Conclusion: No evidence for subject switching off their device prematurely (every subject active throughout 8 hours recording time span)

### Number of walking bouts morning vs afternoon:

In [ ]:
data_wide.value_counts('morning_afternoon')

### Correlation matrix:

In [ ]:
data_wide.head()
data_wide_for_corr = data_wide.copy()

In [ ]:
data_wide_for_corr = data_wide_for_corr.drop('time_stamps', axis=1)
data_wide_for_corr = data_wide_for_corr.drop('morning_afternoon', axis=1)
data_wide_for_corr.head()

In [ ]:
data_wide_num = data_wide_for_corr.iloc[:,5:].copy()

corr_matrix = data_wide_num.corr(method='spearman')

plt.figure(figsize=(10, 10))

# Create a figure and axes with the desired size
fig, ax = plt.subplots(figsize=(10, 10))  # Adjust the figsize to make squares larger

# Plot the matrix using matshow on the created axes
cax = ax.matshow(corr_matrix, cmap="viridis")

# Set ticks
ax.set_xticks(range(len(corr_matrix.columns)))
ax.set_xticklabels(corr_matrix.columns, rotation="vertical")
ax.set_yticks(range(len(corr_matrix.columns)))
ax.set_yticklabels(corr_matrix.columns)


# Add color bar
fig.colorbar(cax)


# Add text annotations
for i in range(len(corr_matrix.columns)):
    for j in range(len(corr_matrix.columns)):
        ax.text(j, i, f"{corr_matrix.iloc[i, j]:.2f}", ha="center", va="center", color="w")

# Display the plot
plt.show()


#### Conclusion: 
The correlations are more or less as expected: E.g. Speed (speed_mean) correlates with stride length (slength_mean) and cadence (cadence_mean). Unfortunately, time_stamps_hours, which indicated the hours after begin of recording does not correlate with any interesting variable.

### Frequency of walking bouts over time:

In [ ]:
data_wide.hist('time_stamps_hours')
plt.xlabel("hours after beginning of recording")
plt.ylabel("frequency of walking bouts")

#### It looks like the frequency of walking bouts decrease over time during the day.

### Frequency of walking bouts over time before vs after rehab:

In [ ]:
data_wide_before_rh = data_wide[data_wide.exp_phase_id == 'T2'].copy()
data_wide_after_rh = data_wide[data_wide.exp_phase_id == 'T3'].copy()

#### Make sure all subjects are present throughout all 8 hours of measuring (no one switched their device off prematurely) before and after rehab:

In [ ]:
time_bins = list(set(data_wide_before_rh["time_stamps_bins"]))
subjects_per_hour = chd.count_unique_subjects_per_hour(data_wide_before_rh, time_bins)
subjects_per_hour

In [ ]:
time_bin_selection = list(range(2,5))
selection_bools = data_wide_before_rh.time_stamps_bins.isin(time_bin_selection)
time_bins_lacking_one_s = data_wide_before_rh.loc[selection_bools]
subjects_2_4 = set(time_bins_lacking_one_s.subject)
len(set(subjects_2_4))

In [ ]:
every_subject = set(data_wide.subject)
every_subject - subjects_2_4

Apparently, subject 8 is missing during time bins 2 - 3.

In [ ]:
time_bins = list(set(data_wide_after_rh["time_stamps_bins"]))
subjects_per_hour = chd.count_unique_subjects_per_hour(data_wide_after_rh, time_bins)
subjects_per_hour

In [ ]:
time_bin_selection = list(range(4,8))
selection_bools = data_wide_after_rh.time_stamps_bins.isin(time_bin_selection)
time_bins_lacking_one_s = data_wide_after_rh.loc[selection_bools]
subjects_4_7 = set(time_bins_lacking_one_s.subject)
len(set(subjects_4_7))

Apparently, subject 8 is missing during time bins 4 - 7.

#### Subject 8 was inactive during 3 hours in the morning before rehab and was inactive during 4 hours in the afternoon after rehab. Did he/she use to take a nap in the morning before rehab and after he/she takes a nap in the afternoon (change of habit)?

In [ ]:
every_subject = set(data_wide.subject)
every_subject - subjects_4_7

In [ ]:
data_wide_before_rh.hist('time_stamps_hours')
plt.xlabel("hours after beginning of recording")
plt.ylabel("frequency of walking bouts")
plt.title('Frequency of walking bouts before rehab:')

In [ ]:
data_wide_after_rh.hist('time_stamps_hours')
plt.xlabel("hours after beginning of recording")
plt.ylabel("frequency of walking bouts")
plt.title('Frequency of walking bouts after rehab:')

#### Conclusion: There seems to be a slight decrease in frequency of walking bouts over time during recording.

### More walking bouts before than after rehab:

In [ ]:
print(data_wide_before_rh.shape)
print(data_wide_after_rh.shape)

In [ ]:
print(len(set(data_wide_before_rh.file_name)))
print(len(set(data_wide_after_rh.file_name)))

#### We cannot draw any conclusions from this fact, as it seems that before rehab patients simply recorded more often (more recording days; see Data_Analysis.ipynb). Since data from all days is pooled, I cannot assess whether or not the number of walking bouts per day changed. 

In [ ]:
data_wide.head()

### K-means clustering to check if I stumble over anything interesting:

In [ ]:
numerical_var_names = list(data_wide.iloc[:,8:].columns)

In [ ]:
# Get the numerical variables:
numerical_variables = data_wide[numerical_var_names]
numerical_variables

#### Scale variables and put them into a numpy array:

In [ ]:
num_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
    ])

In [ ]:
num_attribs = list(numerical_variables)
#cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        #("cat", OneHotEncoder(), cat_attribs),
    ])

#num_vars_prepared = full_pipeline.fit_transform(num_vars_select)
num_vars_prepared = full_pipeline.fit_transform(numerical_variables)

In [ ]:
numerical_variables.shape

In [ ]:
num_vars_prepared.shape

#### Optimal number of clusters not obvious as there is no inertia inflection point:

In [ ]:
inertias = []

for i in range(1,11):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(num_vars_prepared)
    #kmeans.fit(numerical_variables)
    inertias.append(kmeans.inertia_)

plt.plot(range(1,11), inertias, marker='o')
plt.title('Elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=3)
#kmeans.fit(num_data)
kmeans.fit(num_vars_prepared)

In [ ]:
print(num_vars_prepared.shape)
print(numerical_variables.shape)
print(data_wide.shape)

#### Add cluster labels to data frame:

In [ ]:
data_wide_l = data_wide.copy()
data_wide_l['kmeans_labels'] = kmeans.labels_
data_wide_l_T2 = data_wide_l.loc[data_wide_l.exp_phase_id == 'T2',:]
data_wide_l_T3 = data_wide_l.loc[data_wide_l.exp_phase_id == 'T3',:]

#### Clusters do not coincide with phase of experiment (exp_phase_one_hot) and seem to be equally distributed over time:

In [ ]:
index_1 = 6
index_2 = 4
index_3 = 10


print(data_wide_l.columns[index_1])
print(data_wide_l.columns[index_2])
print(data_wide_l.columns[index_3])


plt.figure(figsize=(9, 9))

#X_inverse = rbf_pca.inverse_transform(X_reduced_rbf)

ax = plt.subplot(111, projection='3d')
ax.view_init(10, -70)

scatter = ax.scatter(data_wide_l.iloc[:, index_1], data_wide_l.iloc[:, index_2], 
           data_wide_l.iloc[:, index_3], 
           c=kmeans.labels_, cmap=plt.get_cmap("jet"), 
           marker="o")
ax.set_xlabel(data_wide_l.columns[index_1])
ax.set_ylabel(data_wide_l.columns[index_2])
ax.set_zlabel(data_wide_l.columns[index_3])
#ax.set_xticklabels([])
#ax.set_yticklabels([])
#ax.set_zticklabels([])

# produce a legend with a cross-section of sizes from the scatter
handles, labels = scatter.legend_elements()
legend2 = ax.legend(handles, labels, loc="upper right", title="Clusters")

#save_fig("preimage_plot", tight_layout=False)
plt.show()

In [ ]:
data_wide_l.columns

#### Clusters seem to be based on a somewhat trianguar shaped data distribution:

In [ ]:
index_1 = 11
index_2 = 8
index_3 = 10


print(data_wide_l.columns[index_1])
print(data_wide_l.columns[index_2])
print(data_wide_l.columns[index_3])


plt.figure(figsize=(9, 9))

#X_inverse = rbf_pca.inverse_transform(X_reduced_rbf)

ax = plt.subplot(111, projection='3d')
ax.view_init(10, -70)

scatter = ax.scatter(data_wide_l.iloc[:, index_1], data_wide_l.iloc[:, index_2], 
           data_wide_l.iloc[:, index_3], 
           c=kmeans.labels_, cmap=plt.get_cmap("jet"), 
           marker="o")
ax.set_xlabel(data_wide_l.columns[index_1])
ax.set_ylabel(data_wide_l.columns[index_2])
ax.set_zlabel(data_wide_l.columns[index_3])
#ax.set_xticklabels([])
#ax.set_yticklabels([])
#ax.set_zticklabels([])

# produce a legend with a cross-section of sizes from the scatter
handles, labels = scatter.legend_elements()
legend2 = ax.legend(handles, labels, loc="upper right", title="Clusters")

#save_fig("preimage_plot", tight_layout=False)
plt.show()

In [ ]:
index_1 = 10
index_2 = 8
index_3 = 11


print(data_wide_l.columns[index_1])
print(data_wide_l.columns[index_2])
print(data_wide_l.columns[index_3])


plt.figure(figsize=(9, 9))

#X_inverse = rbf_pca.inverse_transform(X_reduced_rbf)

ax = plt.subplot(111, projection='3d')
ax.view_init(10, -70)

scatter = ax.scatter(data_wide_l.iloc[:, index_1], data_wide_l.iloc[:, index_2], 
           data_wide_l.iloc[:, index_3], 
           c=kmeans.labels_, cmap=plt.get_cmap("jet"), 
           marker="o")
ax.set_xlabel(data_wide_l.columns[index_1])
ax.set_ylabel(data_wide_l.columns[index_2])
ax.set_zlabel(data_wide_l.columns[index_3])
#ax.set_xticklabels([])
#ax.set_yticklabels([])
#ax.set_zticklabels([])

# produce a legend with a cross-section of sizes from the scatter
handles, labels = scatter.legend_elements()
legend2 = ax.legend(handles, labels, loc="upper right", title="Clusters")

#save_fig("preimage_plot", tight_layout=False)
plt.show()

#### The data shape reflects the fact that walking time varies most at a particular mean cadence and stride length standard deviation:

In [ ]:
ax = plt.subplot()
scatter = ax.scatter(data_wide_l.cadence_mean, data_wide_l.WB_time, alpha=0.4,
          c=data_wide_l.kmeans_labels, cmap=plt.get_cmap("jet"))
# produce a legend with a cross-section of sizes from the scatter
handles, labels = scatter.legend_elements()
legend2 = ax.legend(handles, labels, loc="upper right", title="Clusters")
ax.set_xlabel("cadence_mean")
ax.set_ylabel("WB_time")

In [ ]:
ax = plt.subplot()
scatter = ax.scatter(data_wide_l.slength_std, data_wide_l.WB_time, alpha=0.4,
          c=data_wide_l.kmeans_labels, cmap=plt.get_cmap("jet"))
# produce a legend with a cross-section of sizes from the scatter
handles, labels = scatter.legend_elements()
legend2 = ax.legend(handles, labels, loc="upper right", title="Clusters")
ax.set_xlabel("slength_std")
ax.set_ylabel("WB_time")

#### The cluster on the right has higher mean speed values and contrary to the other two clusters seems to show correlation between mean speed and walking bout time:

In [ ]:
ax = plt.subplot()
scatter = ax.scatter(data_wide_l.speed_mean, data_wide_l.WB_time, alpha=0.4,
          c=data_wide_l.kmeans_labels, cmap=plt.get_cmap("jet"))
# produce a legend with a cross-section of sizes from the scatter
handles, labels = scatter.legend_elements()
legend2 = ax.legend(handles, labels, loc="upper right", title="Clusters")
ax.set_xlabel("speed_mean")
ax.set_ylabel("WB_time")

In [ ]:
data_wide_l.plot(kind="scatter", x="cadence_mean", y="speed_mean", alpha=0.4,
    c="kmeans_labels", cmap=plt.get_cmap("jet"), colorbar=False,
    sharex=True)

plt.show()

In [ ]:
ax = plt.subplot()
scatter = ax.scatter(data_wide_l.cadence_mean, data_wide_l.speed_mean, alpha=0.4,
          c=data_wide_l.kmeans_labels, cmap=plt.get_cmap("jet"))
# produce a legend with a cross-section of sizes from the scatter
handles, labels = scatter.legend_elements()
legend2 = ax.legend(handles, labels, loc="upper right", title="Clusters")
ax.set_xlabel("cadence_mean")
ax.set_ylabel("speed_mean")

#### Conclusion: Unfortunately, patients do not seem to group into interesting clusters that would reveal shared characteristics among groups of patients. 